In [1]:
# parse dev data function
def ParseJson(data_json):
    paragraphs = data_json['data'][0]['paragraphs']
    idx_list = list()
    context_list = list()
    ques_list = list()
    ans_list = list()
    for para in paragraphs:
        context = para['context']
        idx = para['qas'][0]['id']
        ques = para['qas'][0]['question']
        ans = '|'.join([ans['text'] for ans in para['qas'][0]['answers']])
        idx_list.append(idx)
        context_list.append(context)
        ques_list.append(ques)
        ans_list.append(ans)
    return idx_list, context_list, ques_list, ans_list

In [4]:
# dev data
import json
dev_file = '/home/t-honli/data/EQnA/dev-v1.1.json' 
dev_data = json.load(open(dev_file, "r"))

In [5]:
import pandas as pd
idx, context, ques, ans = ParseJson(dev_data)
dev_pd = pd.DataFrame({'id':idx, 'context':context, 'question':ques, 'ground_truth':ans}, columns=['id', 'context', 'question', 'ground_truth'])
dev_pd.head(5)

,id,context,question,ground_truth
0,2abdda92eac7f6b73d9b238f503e4219,Start Up & Rev of a 2006 Acura RL SH - AWD.Thi...,2006 acura rl horsepower,290 HP
1,f26402275d1f070d15e951a3ec04dacb,The income tax withholding rate remains at 4.2...,2015 michigan withholding tax rate,4.25 %
2,125674965d00b4087f076223745f1ad5,The experts at Vidal Sassoon bring you salon c...,5rr hair color,Red
3,b153122979cdb753b1466ac8145758c9,Effects depend on the substance : hydrogen per...,acid that burns through skin,nitric acid|hydrogen peroxide
4,f5d4a08ddf0aba6cab1ea9ae6bd1f26b,"Actor Nikolaj Coster - Waldau , the man who pl...",actor who plays jamie lannister,Nikolaj Coster|Waldau


In [6]:
# bidaf ans
import json
import os
file = '/home/t-honli/bi-att-flow/out/EQnA/no_sent_token/01-08-2017/answer/test-020000.json'
bidaf = json.load(open(file, "r"))

In [7]:
dev_pd['bidaf'] = dev_pd.apply(lambda row: bidaf[row['id']], axis=1)

In [33]:
dev_pd['bidaf'] = dev_pd.apply(lambda row: '|||'.join([ phrase+':::'+score for phrase, score in  zip(bidaf[row['id']].split('|||'), str(bidaf['scores'][row['id']]).split('|||'))          ]), axis=1)

In [34]:
pd.set_option('max_colwidth', 200)
dev_pd.head(5)

,id,context,question,ground_truth,bidaf
0,2abdda92eac7f6b73d9b238f503e4219,"Start Up & Rev of a 2006 Acura RL SH - AWD.This has a 3.5L V6 making a decent 290 HP while paired to a 5 Speed Auto.The interior in this was very classy , roomy , & Comfortable .",2006 acura rl horsepower,290 HP,290 HP:::0.37020477652549744
1,f26402275d1f070d15e951a3ec04dacb,"The income tax withholding rate remains at 4.25 % for tax year 2015 . However , the personal exemption amount for tax year 2015 will change to $4,000 . You may continue to use 2014 Michigan Income...",2015 michigan withholding tax rate,4.25 %,4.25 %:::0.945091724395752
2,125674965d00b4087f076223745f1ad5,The experts at Vidal Sassoon bring you salon color expertise with this Vidal Sassoon Pro Series Hair Color in 5RR Medium Vibrant Red . It offers 100 percent gray coverage in a permanent color form...,5rr hair color,Red,gray:::0.5498008131980896
3,b153122979cdb753b1466ac8145758c9,"Effects depend on the substance : hydrogen peroxide removes a bleached layer of skin , while nitric acid causes a characteristic color change to yellow in the skin . Chemical burns may occur throu...",acid that burns through skin,nitric acid|hydrogen peroxide,hydrogen peroxide:::0.4090457260608673
4,f5d4a08ddf0aba6cab1ea9ae6bd1f26b,"Actor Nikolaj Coster - Waldau , the man who plays Jaime Lannister in HBO 's popular television series Game Of Thrones , has been spotted in Hobart , where he has spent time taking in the city 's c...",actor who plays jamie lannister,Nikolaj Coster|Waldau,Waldau:::0.32978367805480957


In [39]:
# cover
def filter_not_cover(row):
    grounds = row['ground_truth'].split('|')
    bidaf = row['bidaf'].split(':::')[0]
    for ground in grounds:
        if bidaf in ground or ground in bidaf:
            return False
    return True
notcover = dev_pd[dev_pd.apply(filter_not_cover, axis=1)]
print ('not cover/total = {}/{}'.format(len(notcover), len(dev_pd)))

not cover/total = 254/2021


In [40]:
# not match
def filter_not_match(row):
    grounds = row['ground_truth'].split('|')
    bidaf = row['bidaf'].split(':::')[0]
    for ground in grounds:
        if bidaf == ground:
            return False
    return True
notmatch = dev_pd[dev_pd.apply(filter_not_match, axis=1)]
print ('not match/total = {}/{}'.format(len(notcover), len(dev_pd)))

not match/total = 254/2021


In [37]:
# cover - match
def filter_cover_not_match(row):
    grounds = row['ground_truth'].split('|')
    bidaf = row['bidaf'].split(':::')[0]
    cover = False
    match = False
    for ground in grounds:
        if bidaf in ground or ground in bidaf:
            cover = True
        if bidaf == ground:
            match = True
    return cover and (not match)
cover_notmatch = dev_pd[dev_pd.apply(filter_cover_not_match, axis=1)]
print ('cover not match/total = {}/{}'.format(len(cover_notmatch), len(dev_pd)))

cover not match/total = 188/2021


In [41]:
cover_notmatch.to_csv('./cover_vs_match/cover_notmatch.tsv', header=True, index=None, sep='\t')
notcover.to_csv('./cover_vs_match/notcover.tsv', header=True, index=None, sep='\t')

In [23]:
dev_pd[dev_pd['question'] == 'what is the zip code for fredericksburg tx']

,id,context,question,ground_truth,bidaf
760,ae192473cf0f4221f7ec19dfeda40604,"Recent home sales , real estate maps , and home value estimator for zip code 78624",what is the zip code for fredericksburg tx,78624,


In [24]:
dev_pd[dev_pd['question'] == 'what time does the tonight show begin']

,id,context,question,ground_truth,bidaf
885,cb345213a804045b5fadbf5c18b58577,"Although the "" Tonight Show "" airs locally at 11 : 35 PM ( on KNBC , Channel 4 ) , the show is actually taped much earlier on the same day , usually from 5 PM to 6 PM , Tuesday through Friday on m...",what time does the tonight show begin,11 : 35 PM,11


In [30]:
dev_pd[dev_pd['question'] == 'how long is your long intestine'].apply(filter_not_cover, axis=1)
notcover[notcover['question'] == 'how long is your long intestine']

,id,context,question,ground_truth,bidaf
